In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], ".."))
from modt.modt import MoDT
from modt._initialization import *
from modt.visualization import *
from modt.utility import *

import pickle
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import normalize
from sklearn.model_selection import RepeatedKFold

In [2]:
datasets = [
    ["breast_cancer_input.np","breast_cancer_target.np"],
    ["iris_input.pd","iris_target.pd"],
    ["steel_input.pd","steel_target.pd"],
    ["abalone_input.pd","abalone_target.pd"], 
    ["contraceptive_input.pd","contraceptive_target.pd"], 
    ["cars_input.pd","cars_target.pd"], 
    ["students_input.pd","students_target.pd"],
    ["adult_input.pd","adult_target.pd"], # Large
    ["bank_input.pd","bank_target.pd"], # Large
    ["hrss_input.pd","hrss_target.pd"], # Large
    ["occupancy_input.pd","occupancy_target.pd"], # Easy
    ["pdm6_input.pd","pdm6_target.pd"], # Easy
    ["banknote_input.pd","banknote_target.pd"], # Easy
    #["sensorless_input.pd","sensorless_target.pd"], # Very Large dataset
]

In [3]:
parameters = {
    "X": None,
    "y": None,
    "n_experts": 3,
    "iterations": 50,
    "max_depth": 2,
    "init_learning_rate": 100,
    "learning_rate_decay": 0.995,
    "initialization_method": "to be overwritten",
    "feature_names": None,
    "class_names": None,
    "use_2_dim_gate_based_on": "to be overwritten",
    "use_2_dim_clustering": False,
    "black_box_algorithm": None,
    }
parameters_fit = {
    "optimization_method": "least_squares_linear_regression",
    "early_stopping": False,
    "use_posterior": False,
    }

In [4]:
gate_reduction_methods = [
    "feature_importance_lr_max",
    None
]

In [5]:
initialization_methods = [
    Random_init(seed=None),
    Kmeans_init(),
    KDTmeans_init(alpha=2.5, beta=0.25, gamma=0.3),
    BGM_init(n_components=parameters["n_experts"],
             mean_precision_prior=0.35,
             weight_concentration_prior_type="dirichlet_process",
             weight_concentration_prior=0.5,
             weight_cutoff=0.0)    
]

In [6]:
start = timer()
runs = 5
rows = []
for dataset in datasets:
    print("Starting",dataset[0],"...")
    data_input = pickle.load(open("../datasets/" + dataset[0], "rb"))
    data_target = pickle.load(open("../datasets/" + dataset[1], "rb"))
    
    use_dataframe = False
    if isinstance(data_input, pd.core.frame.DataFrame):
        use_dataframe = True
        
    dict_results = {
        "dataset" : dataset[0],
        "n_features" : data_input.shape[1]
    } 
    
    for method in gate_reduction_methods:
        for init_method in initialization_methods:
            init_method_name = init_method.__class__.__name__
            
            print("Starting",dataset[0],method,init_method_name,"...")
            parameters["use_2_dim_gate_based_on"] = method
            if parameters["use_2_dim_gate_based_on"] is not None:
                parameters["use_2_dim_clustering"] = True
            else:
                parameters["use_2_dim_clustering"] = False
            parameters["initialization_method"] = init_method

            train_accuracies = []
            val_accuracies = []
            train_accuracies_i = [[] for i in range(parameters["iterations"])]
            
            rkf = RepeatedKFold(n_splits=4, n_repeats=runs)
            for train_idx, val_idx in rkf.split(data_input):
                if use_dataframe:
                    X_temp = data_input.iloc[train_idx]
                    y_temp = data_target.iloc[train_idx]
                    X_temp.reset_index(inplace=True, drop=True)
                    y_temp.reset_index(inplace=True, drop=True)
                else:
                    X_temp = data_input[train_idx]
                    y_temp = data_target[train_idx]

                parameters["X"] = X_temp
                parameters["y"] = y_temp

                modt = MoDT(**parameters)
                modt.fit(**parameters_fit)
                train_accuracies.append(modt.score_internal_disjoint())

                if use_dataframe:
                    X_temp = data_input.iloc[val_idx]
                    y_temp = data_target.iloc[val_idx]
                    X_temp.reset_index(inplace=True, drop=True)
                    y_temp.reset_index(inplace=True, drop=True)
                else:
                    X_temp = data_input[val_idx]
                    y_temp = data_target[val_idx]
                val_accuracies.append(modt.score(X_temp, y_temp))
                for i in range(parameters["iterations"]):
                    train_accuracies_i[i].append(modt.all_accuracies[i])

            train_accuracy = np.mean(train_accuracies)
            val_accuracy = np.mean(val_accuracies)
            train_std = np.std(train_accuracies)
            val_std = np.std(val_accuracies)
            dict_results[str(method) + "_" + str(init_method_name) + "_train"] = train_accuracy
            dict_results[str(method) + "_" + str(init_method_name) + "_test"] = val_accuracy
            dict_results[str(method) + "_" + str(init_method_name) + "_train_std"] = train_std
            dict_results[str(method) + "_" + str(init_method_name) + "_test_std"] = val_std                        

            for i in range(parameters["iterations"]):
                train_i_acc = np.mean(train_accuracies_i[i])
                dict_results[str(method) + "_" + str(init_method_name) + "_train_i_" + str(i)] = modt.all_accuracies[i]
        
    rows.append(dict_results)
    
print("Duration", timer() - start)
df_performance = pd.DataFrame(rows)

Starting breast_cancer_input.np ...
Starting breast_cancer_input.np feature_importance_lr_max Random_init ...
Starting breast_cancer_input.np feature_importance_lr_max Kmeans_init ...
Starting breast_cancer_input.np feature_importance_lr_max KDTmeans_init ...
Starting breast_cancer_input.np feature_importance_lr_max BGM_init ...


C:\Users\Oscar\Anaconda3\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


Starting breast_cancer_input.np None Random_init ...
Starting breast_cancer_input.np None Kmeans_init ...
Starting breast_cancer_input.np None KDTmeans_init ...
Starting breast_cancer_input.np None BGM_init ...
Starting iris_input.pd ...
Starting iris_input.pd feature_importance_lr_max Random_init ...
Starting iris_input.pd feature_importance_lr_max Kmeans_init ...
Starting iris_input.pd feature_importance_lr_max KDTmeans_init ...
Starting iris_input.pd feature_importance_lr_max BGM_init ...
Separation unsuccessful. Gate initialized randomly.
Separation unsuccessful. Gate initialized randomly.
Separation unsuccessful. Gate initialized randomly.
Separation unsuccessful. Gate initialized randomly.
Separation unsuccessful. Gate initialized randomly.
Separation unsuccessful. Gate initialized randomly.
Separation unsuccessful. Gate initialized randomly.
Separation unsuccessful. Gate initialized randomly.
Starting iris_input.pd None Random_init ...
Starting iris_input.pd None Kmeans_init ...

C:\Users\Oscar\Code\mixture-of-decision-trees\benchmarks\..\modt\_initialization.py:91: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=self_modt.n_experts).fit(X)
C:\Users\Oscar\Code\mixture-of-decision-trees\benchmarks\..\modt\_initialization.py:91: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=self_modt.n_experts).fit(X)


Separation unsuccessful. Gate initialized randomly.


...

Separation unsuccessful. Gate initialized randomly.


C:\Users\Oscar\Anaconda3\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
C:\Users\Oscar\Anaconda3\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
C:\Users\Oscar\Anaconda3\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
C:\Users\Oscar\Anaconda3\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarni

Starting pdm6_input.pd None Random_init ...
Starting pdm6_input.pd None Kmeans_init ...
Starting pdm6_input.pd None KDTmeans_init ...
Starting pdm6_input.pd None BGM_init ...
Covariance matrix ill-defined, increasing reg_covar...


C:\Users\Oscar\Anaconda3\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


Separation unsuccessful. Gate initialized randomly.
Starting banknote_input.pd ...
Starting banknote_input.pd feature_importance_lr_max Random_init ...
Starting banknote_input.pd feature_importance_lr_max Kmeans_init ...
Starting banknote_input.pd feature_importance_lr_max KDTmeans_init ...
Starting banknote_input.pd feature_importance_lr_max BGM_init ...
Starting banknote_input.pd None Random_init ...
Starting banknote_input.pd None Kmeans_init ...
Starting banknote_input.pd None KDTmeans_init ...
Starting banknote_input.pd None BGM_init ...
Duration 9035.936561800001


In [7]:
df_performance

,dataset,n_features,feature_importance_lr_max_Random_init_train,feature_importance_lr_max_Random_init_test,feature_importance_lr_max_Random_init_train_std,feature_importance_lr_max_Random_init_test_std,feature_importance_lr_max_Random_init_train_i_0,feature_importance_lr_max_Random_init_train_i_1,feature_importance_lr_max_Random_init_train_i_2,feature_importance_lr_max_Random_init_train_i_3,...,None_BGM_init_train_i_40,None_BGM_init_train_i_41,None_BGM_init_train_i_42,None_BGM_init_train_i_43,None_BGM_init_train_i_44,None_BGM_init_train_i_45,None_BGM_init_train_i_46,None_BGM_init_train_i_47,None_BGM_init_train_i_48,None_BGM_init_train_i_49
0,breast_cancer_input.np,10,0.969653,0.922656,0.005220,0.022506,0.936768,0.950820,0.957845,0.960187,...,0.976581,0.976581,0.976581,0.976581,0.976581,0.976581,0.976581,0.976581,0.976581,0.976581
1,iris_input.pd,4,0.988871,0.957255,0.008865,0.027236,0.955752,0.955752,0.982301,0.973451,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,steel_input.pd,27,0.678690,0.660067,0.019273,0.032515,0.535027,0.536401,0.634615,0.635302,...,0.646978,0.645604,0.644918,0.644918,0.644918,0.644918,0.645604,0.646291,0.646291,0.646978
3,abalone_input.pd,8,0.736957,0.724399,0.008047,0.020851,0.698053,0.698372,0.702841,0.734440,...,0.759974,0.759974,0.759974,0.759974,0.759974,0.759974,0.759655,0.759655,0.759336,0.759336
4,contraceptive_input.pd,9,0.574971,0.549888,0.015438,0.029841,0.472398,0.536652,0.557466,0.553846,...,0.510407,0.512217,0.512217,0.511312,0.512217,0.512217,0.512217,0.511312,0.511312,0.513122
5,cars_input.pd,6,0.795409,0.791551,0.015817,0.019607,0.771605,0.771605,0.771605,0.771605,...,0.893519,0.895833,0.901235,0.895062,0.890432,0.888889,0.890432,0.890432,0.884259,0.895062
6,students_input.pd,11,0.543042,0.463661,0.017282,0.058755,0.502000,0.552000,0.552000,0.552000,...,0.534000,0.534000,0.534000,0.534000,0.534000,0.534000,0.534000,0.534000,0.534000,0.534000
7,adult_input.pd,14,0.839091,0.831702,0.002309,0.018055,0.825922,0.815357,0.837194,0.838343,...,0.825833,0.825833,0.825833,0.825833,0.825833,0.825833,0.825833,0.825833,0.825833,0.825833
8,bank_input.pd,20,0.910781,0.904895,0.002277,0.013145,0.902884,0.905021,0.904244,0.903273,...,0.908517,0.908258,0.908452,0.908193,0.908485,0.908193,0.908485,0.908129,0.908485,0.908193
9,hrss_input.pd,18,0.768208,0.767689,0.003050,0.004247,0.762265,0.762265,0.762265,0.762265,...,0.772415,0.772415,0.772415,0.772415,0.772415,0.772415,0.772415,0.772415,0.772415,0.772415


In [8]:
pickle.dump(df_performance, open("dataframes/df_initialization_methods_lr100.pd", "wb"))